In [1]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [2]:
from pydent import ModelBase

def get_sample(session, x, *args, **kwargs):
    if isinstance(x, int):
        return session.Sample.find(x, *args, **kwargs)
    elif isinstance(x, str):
        return session.Sample.find_by_name(x, *args, **kwargs)
    elif isinstance(x, dict):
        return session.Sample.where(x, *args, **kwargs)
    elif issubclass(x.__class__, ModelBase):
        return x
    
    
def create_new_strain(parent, integrant, qc1, qc2, qcl, project):
    parent = get_sample(aq, parent)
    integrant = get_sample(aq, integrant)

    properties = {
        'Parent': parent,
        'Integrant': integrant,
        'Plasmid': None,
        'Integrated Marker(s)': integrant.properties['Yeast Marker'],
        'Mating Type': parent.properties['Mating Type'],
        'Comp_cell_limit': 'No',
        'Has this strain passed QC?': 'No',
        'Haploids': [],
#         'QC Primer1': get_sample(aq, qc1),
#         'QC Primer2': get_sample(aq, qc2),
#         'QC_length': qcl
    }

    return aq.Sample.new(
        name = parent.name + ' | ' + integrant.name,
        description='',
        sample_type_id=parent.sample_type_id,
        project=project,
        properties=properties
    )

In [18]:
sample = create_new_strain(36492, 36483, None, None, None, "SD2 2021 Redesigns")
exists = aq.Sample.find_by_name(sample.name)
if exists:
    print('exists: {}'.format(exists.id))
else:
    sample.save()
    print(sample.id)

36502


In [19]:
for _, row in df.iterrows():
    parent = aq.Sample.find(int(row['Parent ID']))
    integrant = aq.Sample.find(int(row['DNA ID']))
    new_strain = create_new_strain(parent, integrant, None, None, None, 'SD2 2021 Redesigns')
    existing = aq.Sample.find_by_name(new_strain.name)
    
    if existing:
        print(existing.id)
    else:
        new_strain.save()
        print(new_strain.id)

36476
36399
36477
36478
36479
36480
36481
36400
36401
